In [2]:
%%bash 
wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip

--2020-08-17 02:00:34--  https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.224.136
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.224.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

     0K .......... .......... .......... .......... ..........  0%  850K 79s
    50K .......... .......... .......... .......... ..........  0%  772K 83s
   100K .......... .......... .......... .......... ..........  0%  774K 85s
   150K .......... .......... .......... .......... ..........  0% 87.9M 64s
   200K .......... .......... .......... .......... ..........  0% 98.1M 51s
   250K .......... .......... .......... .......... ..........  0%  784K 57s
   300K .......... .......... .......... .......... ..........  0%  107M 49s
   350K .......... ..........

In [4]:
!apt-get install -y unzip
!unzip creditcardfraud.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 172 kB of archives.
After this operation, 580 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 unzip amd64 6.0-23+deb10u1 [172 kB]
Fetched 172 kB in 0s (10.0 MB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 16492 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-23+deb10u1_amd64.deb ...
Unpacking unzip (6.0-23+deb10u1) ...
Setting up unzip (6.0-23+deb10u1) ...
Processing triggers for mime-support (3.62) ...
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [10]:
!ls -al 

total 282528
drwx------ 8 root nogroup      6144 Aug 17 02:04 .
drwxr-xr-x 1 root root           61 Aug 17 01:59 ..
drwxr-xr-x 4 root root         6144 Aug 17 02:02 .cache
drwxr-xr-x 3 root root         6144 Aug 17 02:02 .config
drwxr-xr-x 2 root root         6144 Aug 17 01:59 .ipynb_checkpoints
drwxr-xr-x 5 root root         6144 Aug 17 01:59 .ipython
drwxr-xr-x 3 root root         6144 Aug 17 01:55 .jupyter
drwxr-xr-x 3 root root         6144 Aug 17 01:54 .local
-rw-r--r-- 1 root root          111 Aug 17 01:54 .yarnrc
-rw-r--r-- 1 root root    150828752 Mar 23  2018 creditcard.csv
-rw-r--r-- 1 root root     69155632 Mar 29  2019 creditcardfraud.zip
-rw-r--r-- 1 root root     69155632 Mar 29  2019 creditcardfraud.zip.1
-rw-r--r-- 1 root root       128854 Aug 17 02:04 downloaddata.ipynb


In [12]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
train_data = df.sample(frac=0.8,random_state=200)
test_data = df.drop(train_data.index)
test_data_no_target = test_data.drop(columns=['Class'])


In [14]:
print('total:{} training:{} testing:{}'.format(df.shape[0], train_data.shape[0], test_data.shape[0]))


total:284807 training:227846 testing:56961


In [15]:
import sagemaker

prefix = 'sagemaker/tutorial-autopilot/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="creditcard.csv", key_prefix=prefix)
print(uri)


s3://sagemaker-us-east-1-230755935769/sagemaker/tutorial-autopilot/input/creditcard.csv


In [16]:
!wget https://raw.githubusercontent.com/imyoungyang/myAWSStudyBlog/master/fraud-dection/3-auto-pilot/auto-pilot-fraud-detection.ipynb 

--2020-08-17 03:48:51--  https://raw.githubusercontent.com/imyoungyang/myAWSStudyBlog/master/fraud-dection/3-auto-pilot/auto-pilot-fraud-detection.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43094 (42K) [text/plain]
Saving to: ‘auto-pilot-fraud-detection.ipynb’

auto-pilot-fraud-de 100%[===================>]  42.08K  --.-KB/s    in 0s      

2020-08-17 03:48:52 (98.8 MB/s) - ‘auto-pilot-fraud-detection.ipynb’ saved [43094/43094]



In [ ]:
import boto3, sys

ep_name = 'tutorial-autopilot-best-model'
sm_rt = boto3.Session().client('runtime.sagemaker')

tn=tp=fn=fp=count=0

with open('bank-additional/bank-additional-full.csv') as f:
    lines = f.readlines()
    for l in lines[1:2000]:   # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[-1]         # Store 'yes'/'no' label
        l = l[:-1]            # Remove label
        l = ','.join(l)       # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))

        if 'yes' in label:
            # Sample is positive
            if 'yes' in response:
                # True positive
                tp=tp+1
            else:
                # False negative
                fn=fn+1
        else:
            # Sample is negative
            if 'no' in response:
                # True negative
                tn=tn+1
            else:
                # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tp/(tp+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))
